# Sales Analysis

In [50]:
import pandas as pd
import os

In [51]:
# Merge 12 months data into single csv file sales.csv

# get the files name in the data/raw directory
files = [file for file in os.listdir("../data/raw")]

# create empty dataframe to store the all months data
all_months_data = pd.DataFrame()

for file in files:
    # read each file in data/raw directory
    current_data = pd.read_csv("../data/raw/"+file)
    # concatenate the all months data frame and currently readed file data
    all_months_data = pd.concat([all_months_data, current_data])

# store the all months data in to data/processed/all_month_sales.csv file without index number
all_months_data.to_csv("../data/processed/all_month_sales.csv", index=False)

In [52]:
all_month_sales = pd.read_csv("../data/processed/all_month_sales.csv")

# Note: dropna(how="any") removes only rows containing actual NaN (missing) values, 
# not invalid strings or datetime errors like NaT (Not a Time). 
# So we use dropna(subset=["Order Date"]) separately to remove rows where the date conversion failed.
all_month_sales = all_month_sales.dropna(how="any")

# read first 5 rows from sales data
all_month_sales.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,209921,USB-C Charging Cable,1,11.95,06/23/19 19:34,"950 Walnut St, Portland, ME 04101"
1,209922,Macbook Pro Laptop,1,1700.0,06/30/19 10:05,"80 4th St, San Francisco, CA 94016"
2,209923,ThinkPad Laptop,1,999.99,06/24/19 20:18,"402 Jackson St, Los Angeles, CA 90001"
3,209924,27in FHD Monitor,1,149.99,06/05/19 10:21,"560 10th St, Seattle, WA 98101"
4,209925,Bose SoundSport Headphones,1,99.99,06/25/19 18:58,"545 2nd St, San Francisco, CA 94016"


## To Use Month based Analysis need to make as correct feature

In [53]:
# Convert 'Order Date' column from string to datetime format.
# The format '%m/%d/%y %H:%M' means: month/day/year hour:minute (e.g., 06/23/19 19:34)
# 'errors="coerce"' ensures that any invalid date values become NaT (Not a Time) instead of throwing an error.
all_month_sales["Order Date"] = pd.to_datetime(all_month_sales["Order Date"], format="%m/%d/%y %H:%M", errors="coerce")

# Remove rows where 'Order Date' could not be converted (NaT values).
# This keeps only valid datetime rows.
all_month_sales = all_month_sales.dropna(subset=["Order Date"])

# Extract only the month number (1–12) from the datetime column.
# Example: '2019-06-23 19:34' → Month = 6
all_month_sales["Month"] = all_month_sales["Order Date"].dt.month.astype(int)

# Display the first 5 rows to verify that the 'Month' column was created correctly.
all_month_sales.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,209921,USB-C Charging Cable,1,11.95,2019-06-23 19:34:00,"950 Walnut St, Portland, ME 04101",6
1,209922,Macbook Pro Laptop,1,1700.0,2019-06-30 10:05:00,"80 4th St, San Francisco, CA 94016",6
2,209923,ThinkPad Laptop,1,999.99,2019-06-24 20:18:00,"402 Jackson St, Los Angeles, CA 90001",6
3,209924,27in FHD Monitor,1,149.99,2019-06-05 10:21:00,"560 10th St, Seattle, WA 98101",6
4,209925,Bose SoundSport Headphones,1,99.99,2019-06-25 18:58:00,"545 2nd St, San Francisco, CA 94016",6


In [54]:
# Convert 'Quantity Ordered' column values from string to integer type
# This ensures we can perform mathematical operations like multiplication.
# Using 'astype(int)' here because the data is already cleaned and guaranteed numeric.
# 'pd.to_numeric(..., errors="coerce")' is safer for messy data but slower,
# since it checks and converts invalid values to NaN. Hence, 'astype()' is preferred for performance.
all_month_sales["Quantity Ordered"] = all_month_sales["Quantity Ordered"].astype(int)

# Convert 'Price Each' column values from string to float type
# Prices can have decimal values, so float is the correct data type.
all_month_sales["Price Each"] = all_month_sales["Price Each"].astype(float)

# Create a new column 'Total Ordered Price' by multiplying quantity and price
# This gives the total sales amount for each individual order.
all_month_sales["Total Orderd Price"] = all_month_sales["Quantity Ordered"] * all_month_sales["Price Each"]

# Display the first 5 rows to confirm the new column is added correctly.
all_month_sales.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Orderd Price
0,209921,USB-C Charging Cable,1,11.95,2019-06-23 19:34:00,"950 Walnut St, Portland, ME 04101",6,11.95
1,209922,Macbook Pro Laptop,1,1700.00,2019-06-30 10:05:00,"80 4th St, San Francisco, CA 94016",6,1700.00
2,209923,ThinkPad Laptop,1,999.99,2019-06-24 20:18:00,"402 Jackson St, Los Angeles, CA 90001",6,999.99
3,209924,27in FHD Monitor,1,149.99,2019-06-05 10:21:00,"560 10th St, Seattle, WA 98101",6,149.99
4,209925,Bose SoundSport Headphones,1,99.99,2019-06-25 18:58:00,"545 2nd St, San Francisco, CA 94016",6,99.99


In [62]:
# Group the dataset by 'Month' and calculate the sum for all numeric columns.
# The 'numeric_only=True' argument ensures only numeric columns (like Quantity, Price, Total) are summed,
# and non-numeric columns (like Product, Address, Order Date) are ignored to prevent TypeError.
all_month_sales.groupby("Month").sum(numeric_only=True)

,Quantity Ordered,Price Each,Total Orderd Price
Month,,,
1,10903,1811768.38,1822256.73
2,13449,2188884.72,2202022.42
3,17005,2791207.83,2807100.38
4,20558,3367671.02,3390670.24
5,18667,3135125.13,3152606.75
6,15253,2562025.61,2577802.26
7,16072,2632539.56,2647775.76
8,13448,2230345.42,2244467.88
9,13109,2084992.09,2097560.13
